# Project 2
#### Problem Statement

> An analysis of paleo-present climate data
>
> * Analyse CO2 vs Temperature Anomaly from 800kyrs – present.
> * Examine one other (paleo/modern) features (e.g. CH4 or polar ice-coverage)
> * Examine Irish context:
>   * Climate change signals: (see Maynooth study: The emergence of a climate change signal in long-term Irish meteorological observations - ScienceDirect)
> * Fuse and analyse data from various data sources and format fused data set as a pandas dataframe and export to csv and json formats
> * For all of the above variables, analyse the data, the trends and the relationships between them (temporal leads/lags/frequency      analysis).
> * Predict global temperature anomaly over next few decades (synthesise data) and compare to published climate models if atmospheric CO2 trends continue
> * Comment on accelerated warming based on very latest features (e.g. temperature/polar-icecoverage)
>
> Use a Jupyter notebook for your analysis and track your progress using GitHub.
>
> Use an academic referencing style


Install xlrd in anaconda to allow pandas to read .xls files
>> conda install -c anaconda xlrd
https://anaconda.org/anaconda/xlrd

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path  

# Download xl files and create csv files
# CO2 data from Luthi et al. 2008 Nature Paper
url1 = 'data/41586_2008_BFnature06949_MOESM31_ESM.xls'
filepath1 = 'data/2008_nature_new_CO2_data.csv'
filepath2 = 'data/2008_nature_Vostok_TD_Dome_C.csv'
filepath3 = 'data/2008_nature_Composite_CO2.csv'
df1=pd.read_excel(url1, '1.  new CO2 data')
df2=pd.read_excel(url1, '2.  Vostok-TD-Dome C')
df3=pd.read_excel(url1, '3.  Composite CO2')

df1.to_csv(filepath1)
df2.to_csv(filepath2)
df3.to_csv(filepath3)

# CO2 data from IPCC Report
url2 = 'data/grl52461-sup-0003-supplementary.xls'
# Temperature Data from Jouzel - epicaDC.deuttemp.EDC3-AICC
url3 = 'data/epicaDC.deuttemp.EDC3-AICC.xls'
# Temperature Data from NOAA
url4 = 'data/edc3deuttemp2007.txt'


ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2
